<a href="https://colab.research.google.com/github/JefteLG/Twitter_Sentiment_Analysis/blob/main/twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1><strong>Twitter Sentiment Analysis<strong><h1></center>

O objetivo desse estudo de caso é criar um modelo que analisa um ou mais Tweets para prever o sentimento(Positivo ou Negativo) presente em cada Tweet. É usado o Processamento de linguagem natural(NLP) juntamente com aprendizagem de maquina para construir esse modelo.

In [ ]:
luska = 'quero @user remover o @user agora'

In [ ]:
newtext = [word for word in luska.split()]


In [ ]:
#Baixar os dados
!wget --no-check-certificate \
    https://raw.githubusercontent.com/JefteLG/Twitter_Sentiment_Analysis/main/Data_Set/twitter.csv \
    -O /tmp/twitter.csv

In [ ]:
tweets_df = pd.read_csv('/tmp/twitter.csv')
tweets_df

In [ ]:
tweet = tweets_df[tweets_df['id']==4]['tweet'].iloc[0]
tweet

In [ ]:
!pip install demoji
import demoji as em

em.download_codes()

In [ ]:
text = 'Olá 😀'

em.findall(text)

In [ ]:
from pprint import pprint
seq = "I bet you didn't know that 😀, 🙋‍♂️, and 🙋‍♀️ are three different emojis."
pprint(seq.encode('unicode-escape'))

(b"I bet you didn't know that \\xf0\\x9f\\x99\\x8b, \\U0001f64b\\u200d\\u2642\\"
 b'ufe0f, and \\U0001f64b\\u200d\\u2640\\ufe0f are three different emojis.')


# Estrutura do Projeto
O projeto prático de Análise de sentimento do Twitter é dividido nas seguintes tarefas:

- Tarefa nº 1: Entender a Declaração do Problema e o caso de negócios.
- Tarefa nº 2: Importar bibliotecas e conjuntos de dados.
- Tarefa nº 3: Executar a análise exploratória de dados.
- Tarefa nº 4: Plotar a nuvem de palavras.
- Tarefa nº 5: Executar a limpeza de dados - remover pontuação.
- Tarefa nº 6: Executar a limpeza de dados - remover palavras de parada(stop words).
- Tarefa nº 7: Executar vetorização de contagem (Tokenization).
- Tarefa nº 8: Criar um pipeline para remover palavras irrelevantes, pontuação e realizar tokenização.
- Tarefa nº 9: Compreender a teoria e a intuição por trás dos classificadores Naive Bayes.
- Tarefa nº 10: Treinar um Classificador Naive Bayes.
- Tarefa nº 11: Avaliar o desempenho do modelo treinado.

## Tarefa nº 1: Entender a Declaração do Problema e o caso de negócios

- Nesse notebook é usado o Processamento de linguagem natural(NLP) juntamente com aprendizagem de maquina para construir um modelo que analisa milhares de Tweets para prever o sentimentos das pessoas.

- A Inteligencia artificial e a analise de sentimentos baseadas em aprendizado de maquina é crucial para empresas, visto que, os insight revelado pela analise visa indicar o grau de qualidade dos serviços e/ou produtos da empresa de acordo com os clientes.(Trabalhos Futuros)

- Esse projeto é diretamente aplicavél a praticamente qualquer empresa que disponhe de meios onlines(Twitter, Instagran, Facebook, WebSite) para interagir com seus clientes.(Trabalhos Futuros)

- Os algoritmos podem ser usados para detectar e possivelmente sinalizar autamaticamente tweets de odio e racismo.(Trabalhos Futuros)

## Tarefa nº 2: Importar bibliotecas e conjuntos de dados

In [ ]:
#Pacotes essenciais para analise numericas, manipulação de data frames e visualização de dados.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Pacote para a criação de nuuvens de palavras na tarefa nº 4
!pip install WordCloud
from wordcloud import WordCloud

In [ ]:
# A base de dados desse exemplo está no meu repositorio do github e pode ser baixada no link abaixo.
!wget --no-check-certificate \
    https://raw.githubusercontent.com/JefteLG/Twitter_Sentiment_Analysis/main/Data_Set/twitter.csv \
    -O /tmp/twitter.csv

In [ ]:
# Utilizando o pandas para ler o arquivo CSV e estrutura-lo em um data frame na variavel `tweets_df` por meio do metodo read_csv().
tweets_df = pd.read_csv('/tmp/twitter.csv')

In [ ]:
# visualização1.
tweets_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [ ]:
# visualização2.
tweets_df

In [ ]:
#Informações sobre a quantidade de Tweets, memoria, tipo de dados e dados faltantes.
tweets_df.info()

In [ ]:
#Exclusão da coluna `id`, não é um dado relevante para o modelo.
tweets_df = tweets_df.drop(columns=['id'], axis=1)

## Tarefa nº 3: Executar a análise exploratória de dados.

In [ ]:
# Esta é uma função no nível dos eixos e desenhará o mapa de calor para os eixos ativos no momento. Nesse caso a função verifica se existe dados faltantes.
sns.heatmap(tweets_df.isnull(), yticklabels=False, cbar=False, cmap="Blues")

In [ ]:
# O método sns.countplot() é usado para mostrar as contagens de observações em cada categoria categórica usando barras(Histograma).
sns.countplot(data=tweets_df, x='label', palette='Set2')
plt.show()

# sns.countplot(tweets_df['label'], label='Count', palette='Set2')
# plt.show()

In [ ]:
# Nova coluna com o tamanho dos tweets
tweets_df['length'] = tweets_df['tweet'].apply(len)

In [ ]:
tweets_df

,label,tweet,length
0,0,@user when a father is dysfunctional and is s...,102
1,0,@user @user thanks for #lyft credit i can't us...,122
2,0,bihday your majesty,21
3,0,#model i love u take with u all the time in ...,86
4,0,factsguide: society now #motivation,39
...,...,...,...
31957,0,ate @user isz that youuu?ðððððð...,68
31958,0,to see nina turner on the airwaves trying to...,131
31959,0,listening to sad songs on a monday morning otw...,63
31960,1,"@user #sikh #temple vandalised in in #calgary,...",67


In [ ]:
# Descobrir o tamanho maximo, minimo e medio dos tweets.
tweets_df['length'].plot(bins=100, kind='hist', figsize=(12,8), color='g')

tweets_df.describe()

In [ ]:
# Selecionar a menor frase
tweets_df[tweets_df['length']==11]['tweet'].iloc[0]

In [ ]:
# Separa o DataFrame em dois Dataframes, um com sentimentos positivos e o outro com sentimentos negativos.
positive_df = tweets_df[tweets_df['label']==0]
negative_df = tweets_df[tweets_df['label']==1]

In [ ]:
positive_df

In [ ]:
negative_df

## Tarefa nº 4: Plotar a nuvem de palavras

In [ ]:
# Criar uma lista de tweets
sentences = tweets_df['tweet'].tolist()

In [ ]:
# O método join() pega todos os itens em um iterável e os une em uma string. é usado o espaço como separador
sentences_as_one_string = " ".join(sentences)

In [ ]:
# Nuvem de palavras de todos os tweets
plt.figure(figsize=(10,10))
plt.imshow(WordCloud().generate(sentences_as_one_string))

In [ ]:
# Nuvem de palavras de todos os tweets NEGATIVOS
negative_sentences = negative_df['tweet'].tolist()
negative_as_one_string = " ".join(negative_sentences)

plt.figure(figsize=(10,10))
plt.imshow(WordCloud().generate(negative_as_one_string))

In [ ]:
# Nuvem de palavras de todos os tweets POSITIVOS
positive_sentences = positive_df['tweet'].tolist()
positive_as_one_string = " ".join(positive_sentences)

plt.figure(figsize=(10,10))
plt.imshow(WordCloud().generate(positive_as_one_string))

## Tarefa nº 5: Executar a limpeza de dados - remover pontuação

In [ ]:
import string

In [ ]:
text = 'Good morning beautiful people :)... I am having fun learning Machine learning and artificial intelligence'

[Artigo sobre a remoção de pontuações de uma string](https://towardsdatascience.com/how-to-efficiently-remove-punctuations-from-a-string-899ad4a059fb)

### Metodo #1

#### Metodo #1.1

In [ ]:
test_punc_remove = [char for char in text if char not in string.punctuation]

In [ ]:
test_punc_remove

In [ ]:
test_punc_remove = ''.join(test_punc_remove)

In [ ]:
test_punc_remove

#### Metodo #1.2

In [ ]:
text_remove_punct = []

In [ ]:
for char in text:
  if char not in string.punctuation:
    text_remove_punct.append(char)

new_text = ''.join(text_remove_punct)

In [ ]:
new_text

### Metodo #2

In [ ]:
punct = string.punctuation + string.digits  

In [ ]:
table_tst = str.maketrans('','',punct)
newtext = text.translate(table_tst)

In [ ]:
newtext

### Metodo #3

In [ ]:
punct = string.punctuation + string.digits  

In [ ]:
table_ = str.maketrans(punct, ' '*len(punct))
newtext = ' '.join(text.translate(table_).split())

In [ ]:
newtext

### Metodo #4

In [ ]:
punct = string.punctuation + string.digits  

In [ ]:
for s in punct:
  text = text.replace(s, '')

In [ ]:
text

### Metodo #5

In [ ]:
import re
newtext = re.sub(r'[^A-Za-z]+', ' ', text)

In [ ]:
newtext

## Tarefa nº 6: Executar a limpeza de dados - remover palavras de parada(stop words)

In [ ]:
#Pacotes para facilitar o processamento de linguagem natural
import nltk
import string

nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stopwords.words('english')

In [ ]:
text = 'Good morning beautiful people :)... I am having fun learning Machine learning and artificial intelligence'

# caracteres para remover
puncts = string.digits + string.punctuation

In [ ]:
#remoção de caracteres
text_remove_punct = [char for char in text if char not in puncts]

In [ ]:
#pega todos os itens em um iterável e os une em uma string.
text_remove_punct = ''.join(text_remove_punct)

In [ ]:
text_remove_punct

In [ ]:
#remoção de stopword
text_remove_stopword = [word for word in text_remove_punct.split() if word.lower() not in stopwords.words('english')]

In [ ]:
text_remove_stopword

## Tarefa nº 7: Executar vetorização de contagem(Tokenization)

## Tarefa nº 8: Criar um pipeline para remover palavras irrelevantes, pontuação e realizar tokenização

## Tarefa nº 9: Compreender a teoria e a intuição por trás dos classificadores Naive Bayes

## Tarefa nº 10: Treinar um Classificador Naive Bayes

## Tarefa nº 11: Avaliar o desempenho do modelo treinado